### 单机多卡训练

<font color="red">我在windows运行命令，一直报错：ValueError: You are going to using gloo on windows, but currently is not supported</font>

In [ ]:
import warnings
warnings.filterwarnings("ignore")

#### 高层API场景

>[paddle.distributed.launch](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/distributed/launch_cn.html#cn-api-distributed-launch)
>
>当调用 paddle.Model 高层API实现训练时，想要启动单机多卡训练非常简单，代码不需要做任何修改，只需要在启动的时候加上参数 -m paddle.distributed.launch

In [ ]:
# 单机单卡启动，默认使用第0号卡
python train.py
# 单机多卡启动，默认使用当前可见的所有的卡
python -m paddle.distributed.launch train.py
# 单机多卡，指定使用第0号卡和第1号卡
python -m paddle.distributed.launch -gpus '0,1' train.py

# 单机多卡，指定使用第0号卡和第1号卡
export CUDA_VISIBLE_DEVICES=0,1
python -m paddle.distributed.launch train.py


#### 基础API场景

使用基础API实现训练，需要修改3处：
>第一处，导入分布式训练所需要的包 import paddle.distributed as dist
>
>第二处，初始化并行环境 dist.init_parallel_env()
>
>第三处，用 paddle.DataParallel(mnist) 封装模型
>

In [ ]:
import paddle
# 第一步，导入分布式计算包
import paddle.distributed as dist
from paddle.vision.transforms import ToTensor

# 第二步，初始化运行环境
dist.init_parallel_env()
# 加载数据集
train_dataset = paddle.vision.MNIST(mode="train",transform=ToTensor())
test_dataset = paddle.vision.MNIST(mode="test",transform=ToTensor())
# 模型组网
mnist = paddle.nn.Sequential(
    paddle.nn.Flatten(1,-1),
    paddle.nn.Linear(784,512),
    paddle.nn.ReLU(),
    paddle.nn.Dropout(0.2),
    paddle.nn.Linear(512,10)
)
# 用 DataLoader 加载数据
train_loader = paddle.io.DataLoader(train_dataset,batch_size=32,shuffle=True)

# 第三步，用paddle.DataParallel 封装模型
mnist = paddle.DataParallel(mnist)

mnist.train()


epochs = 5
# 优化器
optim = paddle.optimizer.Adam(parameters=mnist.parameters())

for epoch in range(epochs):
    for batch_id,data in enumerate(train_loader()):
        x_data = data[0]
        y_data = data[1]
        predicts = mnist(x_data)

        # 损失函数
        loss = paddle.nn.functional.cross_entropy(predicts,y_data)
        # 准确率
        acc = paddle.metric.accuracy(predicts,y_data)
        # 反向传播更新参数，并清除之前的梯度信息
        loss.backward()
        optim.step()
        optim.clear_grad()

        # 打印信息
        if (batch_id+1 % 900) == 0:
            print(f"epoch：{epoch}，batch_id：{batch_id}，loss：{loss.numpy()}，acc：{acc.numpy()}")







#### spawn启动

In [ ]:
import paddle
import paddle.nn as nn
import paddle.optimizer as opt
import paddle.distributed as dist

In [ ]:
from __future__ import print_function

import paddle
import paddle.nn as nn
import paddle.optimizer as opt
import paddle.distributed as dist

class LinearNet(nn.Layer):
    def __init__(self):
        super(LinearNet, self).__init__()
        self._linear1 = nn.Linear(10, 10)
        self._linear2 = nn.Linear(10, 1)

    def forward(self, x):
        return self._linear2(self._linear1(x))

def train(print_result=False):

    # 1. 初始化并行训练环境
    dist.init_parallel_env()

    # 2. 创建并行训练 Layer 和 Optimizer
    layer = LinearNet()
    dp_layer = paddle.DataParallel(layer)

    loss_fn = nn.MSELoss()
    adam = opt.Adam(
        learning_rate=0.001, parameters=dp_layer.parameters())

    # 3. 运行网络
    inputs = paddle.randn([10, 10], 'float32')
    outputs = dp_layer(inputs)
    labels = paddle.randn([10, 1], 'float32')
    loss = loss_fn(outputs, labels)

    if print_result is True:
        print("loss:", loss.numpy())

    loss.backward()

    adam.step()
    adam.clear_grad()

# 使用方式1：仅传入训练函数
# 适用场景：训练函数不需要任何参数，并且需要使用所有当前可见的GPU设备并行训练
if __name__ == '__main__':
    dist.spawn(train)

# 使用方式2：传入训练函数和参数
# 适用场景：训练函数需要一些参数，并且需要使用所有当前可见的GPU设备并行训练
if __name__ == '__main__':
    dist.spawn(train, args=(True,))

# 使用方式3：传入训练函数、参数并指定并行进程数
# 适用场景：训练函数需要一些参数，并且仅需要使用部分可见的GPU设备并行训练，例如：
# 当前机器有8张GPU卡 {0,1,2,3,4,5,6,7}，此时会使用前两张卡 {0,1}；
# 或者当前机器通过配置环境变量 CUDA_VISIBLE_DEVICES=4,5,6,7，仅使4张
# GPU卡可见，此时会使用可见的前两张卡 {4,5}
if __name__ == '__main__':
    dist.spawn(train, args=(True,), nprocs=2)

# 使用方式4：传入训练函数、参数、指定进程数并指定当前使用的卡号
# 使用场景：训练函数需要一些参数，并且仅需要使用部分可见的GPU设备并行训练，但是
# 可能由于权限问题，无权配置当前机器的环境变量，例如：当前机器有8张GPU卡
# {0,1,2,3,4,5,6,7}，但你无权配置CUDA_VISIBLE_DEVICES，此时可以通过
# 指定参数 gpus 选择希望使用的卡，例如 gpus='4,5'，
# 可以指定使用第4号卡和第5号卡
if __name__ == '__main__':
    dist.spawn(train, nprocs=2, gpus='4,5')
